In [ ]:
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,Input,TimeDistributed,LSTM
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D , Conv3D
from keras.optimizers import SGD
from keras import optimizers
from tensorflow.python import keras
from tensorflow.python.keras.applications import vgg16,resnet

EPOCHS = 10
BATCH_SIZE = 32
Train_Videos_dir = '/content/drive/My Drive/Training_set/Training'
Test_Videos_dir = '/content/drive/My Drive/Testing_set'

def Make_submission(predictions):
    results = []
    Myvideos = []

    for pred in predictions:
    #np.argmax return the index of the max element in the array
        if np.argmax(pred) == 0:
            results.append('2')
        elif np.argmax(pred) == 1:
            results.append('0')
        elif np.argmax(pred) == 2:
            results.append('1')
        elif np.argmax(pred) == 3:
            results.append('3')
        elif np.argmax(pred) == 4:
            results.append('4')

    Videos = np.load('testvideos.npy')
    for vid in Videos:
        Myvideos.append(vid)

    submission = pd.DataFrame({
        "Videos": Myvideos,
        "Label": results
    })
    submission.to_csv('Submit.csv', index=False)
    print('Done Saving Data You Can now Submit')

#encoding based on the training data
def Encoding_data(Folder_name):
    if Folder_name == 'Basketball':
        return np.array([1, 0, 0, 0, 0])
    if Folder_name == 'Diving':
        return np.array([0, 1, 0, 0, 0])
    if Folder_name == 'Jumping':
        return np.array([0, 0, 1, 0, 0])
    if Folder_name == 'Tennis':
        return np.array([0, 0, 0, 1, 0])
    if Folder_name == 'Walking':
        return np.array([0, 0, 0, 0, 1])

#takes directory of the data and return array of [frames, label]
def read_train_videos(Directory):
    All_videos = []
    All_Labels = []
    for fold in os.listdir(Directory):
        #Categorical enconding
        Label = Encoding_data(fold)
        Folder_path = os.path.join(Directory, fold)
        for vid in tqdm(os.listdir(Folder_path)):
            Myvideos = []
            Video_path = os.path.join(Folder_path, vid)
            cap = cv2.VideoCapture(Video_path)
            nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            rem = nframe
            cnt = 0
            #iterates on 1 video
            while rem >= 25:
                ret, frame = cap.read()
                if frame is None:
                    break
                if cnt < 25:
                    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    final_frame = cv2.resize(frame, (100, 100))
                   
                    #final_frame=np.reshape(final_frame,(128,128,1))

                    Myvideos.append(final_frame)
                    cnt = cnt + 1
                else:
                    cnt = 0
                    All_videos.append(Myvideos)
                    All_Labels.append(Label)
                    Myvideos = []
                    rem = rem - 25

    FinalVideos, FinalLabels =  shuffle(All_videos, All_Labels)
    np.save("All_Videos.npy", FinalVideos)
    np.save('Labels.npy', FinalLabels)
    return FinalVideos, FinalLabels

def read_test_videos(Directory):
    All_videos = []
    array_indicies=[]
    total_sub_videos=0

    for vid in tqdm(os.listdir(Directory)):
        MyvideosList = []
        Video_path = os.path.join(Directory, vid)
        cap = cv2.VideoCapture(Video_path)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        rem = nframe
        cnt = 0
        while rem>=25:
            ret, frame = cap.read()
            if frame is None:
                break
            if cnt<25:
                #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                final_frame = cv2.resize(frame, (100, 100))
                #final_frame=np.reshape(final_frame,(128,128,1))
                MyvideosList.append(final_frame)
                cnt=cnt+1
            else:
                cnt = 0
                All_videos.append(MyvideosList)
                MyvideosList = []
                total_sub_videos=total_sub_videos+1
                rem = rem - 25
        array_indicies.append(total_sub_videos) 
        total_sub_videos=0     
        
        

    np.save('array_indicies.npy',array_indicies) 
    np.save('Testvideos.npy',All_videos)
    return All_videos,array_indicies


def get_model():
    """ Return the Keras model of the network
    """
    
    model = Sequential()
    # 1st layer group
    model.add(Conv3D(64,( 3, 3, 3), activation='relu',
                             name='conv1',
                            input_shape=(25, 100,100,3),padding="same",strides=(1, 1, 1)) )

    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                            name='pool1',padding="valid"))
    # 2nd layer group
    model.add(Conv3D(128,( 3, 3, 3), activation='relu',
                             name='conv2',
                            padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                           name='pool2',padding="valid"))
    # 3rd layer group
    model.add(Conv3D(256, (3, 3, 3), activation='relu',
                             name='conv3a',
                            padding="same"))
    model.add(Conv3D(256, (3, 3, 3), activation='relu',
                             name='conv3b',
                            padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                            name='pool3',padding="valid"))
    # 4th layer group
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                             name='conv4a',
                            padding="same"))
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                             name='conv4b',
                            padding="same"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                            name='pool4',padding="valid"))
    # 5th layer group
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                             name='conv5a',
                            padding="same"))
    model.add(Conv3D(512, (3, 3, 3), activation='relu',
                             name='conv5b',
                            padding="same"))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                            name='pool5',padding="valid"))
    model.add(Flatten())
   
    model.add(Dense(5, activation='softmax', name='fc8'))


    return model

def run_model(Model, TrainData, TrainY, TestData):
    opt = optimizers.adam(lr=0.0001)
    #optimizer='adam'
    Model.compile(loss="categorical_crossentropy", optimizer= opt , metrics=["accuracy"])
    print("aa")
    print(np.shape(TrainData))
    print(np.shape(TrainY))
    #TrainData=TrainData.reshape(TrainData.shape[0],224,224,25,1)
    print(np.shape(TrainData))
    Model.fit(np.array(TrainData), np.array(TrainY), epochs=4, verbose=1, batch_size=10)
    print("Done training")
    predictions = Model.predict(np.array(TestData))
    print("Done testing")

    return predictions

if __name__ == '__main__':
    if os.path.exists('/content/drive/My Drive/All_Videoss.npy'):
        TrainData = np.load('/content/drive/My Drive/All_Videos2.npy')
        Mylabels =  np.load('/content/drive/My Drive/Labels2.npy')
        TestData = np.load('/content/drive/My Drive/Testvideos2.npy',allow_pickle=True)
        indicies=np.load('/content/drive/My Drive/array_indicies2.npy')
    else:
        TrainData, Mylabels = read_train_videos(Train_Videos_dir)
        TestData,indicies = read_test_videos(Test_Videos_dir)

    print(np.shape(TestData))
    M = get_model()
    if os.path.exists('predic.npy'):
        Predictions=np.load('predic.npy')
    else:
        Predictions = run_model(M, TrainData, Mylabels, TestData)
        np.save('predic.npy', Predictions)
    
    ind=0
  
    
    results=[]
    
    for i in range(0,len(indicies)):
        freq_arr=[0,0,0,0,0]
        for j in range(0,indicies[i]):
            pred=Predictions[ind+j]
            
            if np.argmax(pred) == 0:
                freq_arr[2]=freq_arr[2]+1
            elif np.argmax(pred) == 1:
                freq_arr[0]=freq_arr[0]+1
            elif np.argmax(pred) == 2:
                freq_arr[1]=freq_arr[1]+1
            elif np.argmax(pred) == 3:
                freq_arr[3]=freq_arr[3]+1
            elif np.argmax(pred) == 4:
                freq_arr[4]=freq_arr[4]+1

        results.append(np.argmax(freq_arr))
        

        ind=ind+indicies[i]


    #np.save('predic.npy', Predictions)
    #Make_submission(Predictions)
    videos_names=[]
    for vid in os.listdir(Test_Videos_dir):
        videos_names.append(vid)

    submission = pd.DataFrame({
        "Video": videos_names,
        "Label": results
    })
    submission.to_csv('Submit.csv', index=False)
    print('Done Saving Data You Can now Submit')
    
